In [1]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers
from sklearn.svm import SVC
import math

In [3]:
train_data = pd.read_csv("./data/train.csv", header=None).to_numpy()
train_whole_X = train_data[:, :-1]
train_whole_Y = train_data[:, -1:].astype(int)

#### Scaling train data

In [4]:
train_whole_X = train_whole_X/255

#### Choosing data points

In [5]:
train_X = train_whole_X[(train_whole_Y[:, 0]==6) | (train_whole_Y[:, 0]==7)]

train_Y = train_whole_Y[(train_whole_Y[:, 0]==6) | (train_whole_Y[:, 0]==7)]

m = len(train_X)

train_Y_c = np.array([1 if i[0]==6 else -1 for i in train_Y]).reshape((m, 1))

# Part 1

## Part 1.a

#### calculating values of $\alpha$

In [5]:
LP = matrix((train_Y_c @ train_Y_c.T)*(train_X @ train_X.T), tc = 'd')

LQ = matrix(np.ones(m).reshape((m, 1))* -1, tc = 'd')

LG = np.zeros((2*m, m))
for i in range(m):
    LG[2*i][i] = 1
    LG[2*i+1][i] = -1
LG = matrix(LG, tc = 'd')

LH = np.ones(2*m)
for i in range(m):
    LH[2*i+1] = 0
LH = matrix(LH, tc = 'd')

LA = matrix(train_Y_c.T, tc = 'd')

LB = matrix(np.array([0]).reshape((1,1)), tc = 'd')

In [6]:
l_sol = solvers.qp(LP, LQ, LG, LH, LA, LB)

     pcost       dcost       gap    pres   dres
 0: -1.8559e+02 -7.3411e+03  4e+04  2e+00  1e-12
 1: -1.0512e+02 -3.6276e+03  7e+03  3e-01  9e-13
 2: -3.0675e+01 -8.9272e+02  1e+03  6e-02  6e-13
 3: -5.5085e+00 -2.2083e+02  3e+02  1e-02  2e-13
 4: -1.4249e+00 -4.5788e+01  7e+01  2e-03  4e-14
 5: -2.9569e-01 -1.3553e+01  2e+01  5e-04  2e-14
 6: -2.0996e-01 -3.9259e+00  5e+00  1e-04  1e-14
 7: -1.0278e-01 -3.5936e+00  4e+00  8e-05  9e-15
 8: -2.5356e-01 -2.0088e+00  2e+00  3e-05  8e-15
 9: -2.0734e-01 -1.6993e+00  2e+00  1e-05  8e-15
10: -4.0852e-01 -1.0189e+00  6e-01  3e-06  8e-15
11: -4.8680e-01 -7.8876e-01  3e-01  2e-16  8e-15
12: -5.3127e-01 -7.1798e-01  2e-01  2e-16  7e-15
13: -5.9184e-01 -6.2963e-01  4e-02  2e-16  8e-15
14: -6.0475e-01 -6.1102e-01  6e-03  2e-16  8e-15
15: -6.0763e-01 -6.0779e-01  2e-04  2e-16  8e-15
16: -6.0771e-01 -6.0771e-01  2e-06  2e-16  8e-15
17: -6.0771e-01 -6.0771e-01  3e-08  2e-16  8e-15
Optimal solution found.


In [7]:
l_sol = np.array(l_sol['x'])

In [8]:
l_alpha = l_sol[l_sol>1e-5].reshape((-1, 1))
l_sv_y = train_Y_c[l_sol>1e-5].reshape((-1, 1))
l_sv_x = train_X[(l_sol>1e-5).reshape((-1,))]

#### calculating weights

In [9]:
l_w = (l_alpha * l_sv_y * l_sv_x).sum(axis=0).reshape((-1, 1))

In [10]:
l_w.shape

(784, 1)

#### calculating intercept

In [11]:
a1 = []
a2 = []
for i in range(len(l_sv_y)):
    if l_sv_y[i]==1:
        a1.append((l_sv_x[i]@l_w))
    else:
        a2.append((l_sv_x[i]@l_w))
l_b = 0.5*(min(a1) + max(a2))

In [12]:
l_b

array([0.01026643])

In [13]:
def predict(X, w, b):
    return np.array([1 if (i@w + b) >= 0 else -1 for i in X])

In [14]:
def getAccuracy(original, predicted):
    return (sum([1 if original[i]==predicted[i] else 0 for i in range(len(original))])/len(original))*100

#### Predicting on training data

In [15]:
train_linear_pred = predict(train_X, l_w, l_b)

In [16]:
getAccuracy(train_Y_c, train_linear_pred)

100.0

### read validation data

In [6]:
val_data = pd.read_csv("./data/val.csv", header=None).to_numpy()
val_whole_X = val_data[:, :-1]
val_whole_Y = val_data[:, -1:].astype(int)

#### scaling validation data

In [7]:
val_whole_X = val_whole_X/255.0

#### Choosing data points

In [8]:
val_X = val_whole_X[(val_whole_Y[:, 0]==6) | (val_whole_Y[:, 0]==7)]

val_Y = val_whole_Y[(val_whole_Y[:, 0]==6) | (val_whole_Y[:, 0]==7)]

val_Y_c = np.array([1 if i[0]==6 else -1 for i in val_Y]).reshape((-1,1))

#### Predicting on validation data

In [20]:
val_linear_pred = predict(val_X, l_w, l_b)

In [21]:
getAccuracy(val_Y_c, val_linear_pred)

100.0

### read testing data

In [9]:
test_data = pd.read_csv("./data/test.csv", header=None).to_numpy()
test_whole_X = test_data[:, :-1]
test_whole_Y = test_data[:, -1:].astype(int)

#### scaling test data

In [10]:
test_whole_X = test_whole_X/255.0

#### choosing data points

In [11]:
test_X = test_whole_X[(test_whole_Y[:, 0]==6) | (test_whole_Y[:, 0]==7)]

test_Y = test_whole_Y[(test_whole_Y[:, 0]==6) | (test_whole_Y[:, 0]==7)]

test_Y_c = np.array([1 if i[0]==6 else -1 for i in test_Y]).reshape((-1, 1))

#### Predicting on testing data

In [25]:
test_linear_pred = predict(test_X, l_w, l_b)

In [26]:
getAccuracy(test_Y_c, test_linear_pred)

100.0

## Part 1.b

In [12]:
from tqdm import tqdm

In [13]:
def kernel(x, z):
    return math.exp(-0.05 * (((x - z)**2).sum()))

In [34]:
Pt = np.diag(np.ones(m))
for i in tqdm(range(m)):
    for j in range(i+1, m):
        Pt[i][j] = Pt[j][i] = kernel(train_X[i], train_X[j])

100%|██████████| 4500/4500 [01:12<00:00, 61.80it/s] 


In [14]:
from scipy.spatial.distance import cdist

Pt = np.exp(-0.05*(cdist(train_X, train_X, 'euclidean')**2))

In [30]:
filename = "P.pkl"
file = open(filename, "wb")
pickle.dump(Pt, file)
file.close()

NameError: name 'pickle' is not defined

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
Pt1 = cdist(train_X, train_X)

In [ ]:
Pt1 = np.exp(-0.05*np.square(Pt1))

In [ ]:
Pt == Pt1

In [20]:
(train_Y_c @ train_Y_c.T).shape

(4500, 4500)

In [15]:
yyT = (train_Y_c @ train_Y_c.T)
LP = matrix(yyT * Pt, tc = 'd')

LQ = matrix(np.zeros(m).reshape((m, 1))-1, tc = 'd')

LG = np.zeros((2*m, m))
for i in range(m):
    LG[2*i][i] = 1
    LG[2*i+1][i] = -1
LG = matrix(LG, tc = 'd')

LH = np.ones(2*m)
for i in range(m):
    LH[2*i+1] = 0
LH = matrix(LH, tc = 'd')

LA = matrix(train_Y_c.T, tc = 'd')

LB = matrix(np.array([0]).reshape((1,1)), tc = 'd')

In [16]:
l_sol = solvers.qp(LP, LQ, LG, LH, LA, LB)

     pcost       dcost       gap    pres   dres
 0: -1.1028e+02 -7.5103e+03  4e+04  2e+00  2e-15
 1: -5.6897e+01 -3.8143e+03  7e+03  3e-01  2e-15
 2: -1.3830e+01 -9.2363e+02  1e+03  4e-02  3e-15
 3: -3.9528e+01 -1.9616e+02  2e+02  4e-03  2e-15
 4: -5.5680e+01 -1.2195e+02  7e+01  2e-03  2e-15
 5: -6.4887e+01 -8.8241e+01  2e+01  1e-04  1e-15
 6: -6.9418e+01 -7.6829e+01  7e+00  2e-05  1e-15
 7: -7.1201e+01 -7.3118e+01  2e+00  2e-06  1e-15
 8: -7.1797e+01 -7.2062e+01  3e-01  1e-07  1e-15
 9: -7.1900e+01 -7.1908e+01  8e-03  2e-09  1e-15
10: -7.1903e+01 -7.1903e+01  1e-04  3e-11  1e-15
11: -7.1903e+01 -7.1903e+01  2e-06  3e-13  1e-15
Optimal solution found.


In [17]:
l_alpha = np.array(l_sol['x'])

In [ ]:
import pickle
filename = "alphas.pkl"
file = open(filename, "wb")
pickle.dump(l_sol, file)
file.close()

In [18]:
sum(l_alpha > 1e-5)

array([678])

In [19]:
indices = np.where(l_alpha >1e-5)[0]

In [21]:
indices.shape

(678,)

In [22]:
l_alpha = l_alpha[indices]
l_sv_x = train_X[indices]
l_sv_y = train_Y_c[indices]

In [23]:
l_alpha.shape

(678, 1)

In [41]:
# l_alpha = []
# l_sv_x = []
# l_sv_y = []
# for i in indices:
#     l_alpha.append(l_sol[i])
#     l_sv_x.append(train_X[i])
#     l_sv_y.append(train_Y_c[i])

In [24]:
len(l_alpha)

678

In [42]:
# l_alpha = np.array(l_alpha)
# l_sv_y = np.array(l_sv_y)
# l_sv_x = np.array(l_sv_x)
# len(l_alpha)

678

In [25]:
Pt1_sv = cdist(l_sv_x, l_sv_x)
Pt1_sv = np.exp(-0.05*(np.square(Pt1_sv)))

In [44]:
Pt_sv = np.diag(np.ones(678))
for i in tqdm(range(678)):
    for j in range(i+1, 678):
        Pt_sv[i][j] = Pt_sv[j][i] = kernel(l_sv_x[i], l_sv_x[j])

100%|██████████| 678/678 [00:01<00:00, 429.35it/s]


In [26]:
a1 = []
a2 = []
t = l_alpha * l_sv_y
for i in tqdm(range(678)):
    if l_sv_y[i]==1:
        a1.append(Pt1_sv[i]@t)
    else:
        a2.append(Pt1_sv[i]@t)
l_b = 0.5*(min(a1) + max(a2))

100%|██████████| 678/678 [00:00<00:00, 257818.51it/s]


In [27]:
l_b

array([-0.60416892])

In [34]:
t = l_alpha * l_sv_y
# print(t.shape)
def _predict(x):
    l = np.array([kernel(x, z) for z in l_sv_x]).reshape((-1, 1))
    l = t * l
    if sum(l) >= 0:
        return 1
    else:
        return -1

def predict(X):
    return np.array([_predict(x) for x in X])

#### predicting on validation data

In [35]:
sum(predict(val_X) == val_Y_c.ravel())/5

99.8

In [62]:
val_gk_pred 

array([-1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1,  1, -1, -1,  1,
       -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1,
        1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,
       -1, -1, -1,  1, -1

In [63]:
sum(val_gk_pred == val_Y_c.ravel())/5

78.8

In [64]:
getAccuracy(val_Y_c, val_gk_pred)

78.8

#### predicting on test data

In [123]:
test_gk_pred = predict(test_X, train_X, train_Y_c, gk_b)

NameError: name 'gk_b' is not defined

In [64]:
getAccuracy(test_Y_c, test_gk_pred)

50.0

## Part 1.c

### Sklearn Linear SVM

In [116]:
linearModel = SVC(kernel='linear')

In [117]:
linearModel.fit(train_X, train_Y_c)

/home/mcs20192566/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

#### predicting on training data

In [ ]:
train_skl_pred = linearModel.predict(train_X)

In [ ]:
getAccuracy(train_Y_c, train_skl_pred)

#### predicting on validation data

In [ ]:
val_skl_pred = linearModel.predict(val_X)

In [ ]:
getAccuracy(val_Y_c, val_skl_pred)

#### predicting on test data

In [ ]:
test_skl_pred = linearModel.predict(test_X)

In [ ]:
getAccuracy(test_Y_c, test_skl_pred)

#### comparing number of support vectors

In [118]:
print('Number of support vectors in sklearn model = ', linearModel.n_support_.sum())

Number of support vectors in sklearn model =  52


In [ ]:
print('Number of support vectors in cvxopt model = ', sum(linear_alpha > 0))

#### comparing weight(w) values

In [ ]:
print('Weight values in sklearn model = ', linearModel.coef_[0])

In [ ]:
print('Weight values in cvxopt model = ', linear_w)

In [ ]:
print('difference between cvxopt weights and sklearn weights = ', linear_w - linearModel.coef_[0])

#### comparing intercept value

In [ ]:
print('Intercept value in sklearn model = ', linearModel.intercept_[0])

In [ ]:
print('Intercept value in cvxopt model = ', linear_b)

### Sklearn Gaussian kernel SVM

In [ ]:
gkmodel = SVC(gamma = 0.05)

In [ ]:
gkmodel.fit(mt_X, mt_Y_c)

#### predicting on training data

In [ ]:
train_skg_pred = gkModel.predict(train_X)

In [ ]:
getAccuracy(train_Y_c, train_skg_pred)

#### predicting on validation data

In [ ]:
val_skg_pred = gkModel.predict(val_X)

In [ ]:
getAccuracy(val_Y_c, val_skg_pred)

#### predicting on test data

In [ ]:
test_skg_pred = linearModel.predict(test_X)

In [ ]:
getAccuracy(test_Y_c, test_skg_pred)

#### comparing number of support vectors

In [ ]:
print('Number of support vectors in sklearn model = ', gkModel.n_support_.sum())

In [ ]:
print('Number of support vectors in cvxopt model = ', sum(gk_alpha > 0))

#### comparing intercept value

In [ ]:
print('Intercept value in sklearn model = ', gkModel.intercept_[0])

In [ ]:
print('Intercept value in cvxopt model = ', gk_b)